In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas tqdm matplotlib

    100% |████████████████████████████████| 10.0MB 153kB/s eta 0:00:01   61% |███████████████████▊            | 6.2MB 18.6MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 9.4MB/s eta 0:00:01
    100% |████████████████████████████████| 13.0MB 121kB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 3.1MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 8.8MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 7.8MB/s eta 0:00:01
You are using pip version 9.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import os
import numpy as np
#import pandas as pd
import cv2
#from tqdm import tqdm
from matplotlib import pyplot as plt 

In [4]:
class FaceDetector:
    def __init__(self, 
                 modelFile="../models/res10_300x300_ssd_iter_140000.caffemodel",
                 configFile = "../models/deploy.prototxt",
                 max_interations = 300,
                 conf_threshold = 0.60,
                 normalized_dim = (32,32)):
        self.modelFile = modelFile
        self.configFile = configFile
        self.max_interations = max_interations
        self.conf_threshold = conf_threshold
        self.normalized_dim = normalized_dim
        self.net = cv2.dnn.readNetFromCaffe(self.configFile, self.modelFile)
        
    def extract_random_faces(self, filename, num_faces):
        captured_faces = []
        iterations = 0
        v_cap = cv2.VideoCapture(filename)
        v_length = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while len(captured_faces) < num_faces and iterations < self.max_interations:
            iterations += 1
            v_cap.set(1, np.random.randint(v_length)-1)
            
            ret, img = v_cap.read()
        
            if ret == True:
                (h, w) = img.shape[:2]
                blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (103.93, 116.77, 123.68))
                self.net.setInput(blob)
                detections = self.net.forward()
                for i in range(detections.shape[2]):
                    confidence = detections[0, 0, i, 2]            
                    if confidence > self.conf_threshold:
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (x1, y1, x2, y2) = box.astype("int")
                        face=img[y1:y2, x1:x2]
                        
                        # normlize
                        face = cv2.resize(face, self.normalized_dim)
                        captured_faces.append(face)
        
        # When everything done, release the video capture and video write objects
        v_cap.release()
    
        return captured_faces

In [7]:
df = pd.DataFrame()
for root, dirs, files in os.walk('../videos', topdown=False):
    for name in dirs:
        print(name)
        dfdir = pd.read_json('../videos/' + name + '/metadata.json')
        dfdir = dfdir.T
        dfdir['dir'] = name
        df = df.append(dfdir)

dfdc_train_part_11


In [8]:
df.describe()

,label,original,split,dir
count,2118,1759,2118,2118
unique,2,359,1,1
top,FAKE,qifhccqwpi.mp4,train,dfdc_train_part_11
freq,1759,30,2118,2118


In [9]:
df.sample(10)

,label,original,split,dir
envmivjahd.mp4,FAKE,dqavjmidxr.mp4,train,dfdc_train_part_11
ynsxzcicxv.mp4,REAL,NaN,train,dfdc_train_part_11
mesyfmgozo.mp4,FAKE,wotgiryojo.mp4,train,dfdc_train_part_11
gwnxqtenem.mp4,FAKE,zjhmmibena.mp4,train,dfdc_train_part_11
vnzfjmxalx.mp4,FAKE,jrzsengxux.mp4,train,dfdc_train_part_11
torlpqwpfj.mp4,FAKE,oimivjeigb.mp4,train,dfdc_train_part_11
jbbysdmnwr.mp4,FAKE,iohuzwvefi.mp4,train,dfdc_train_part_11
umdojzbwds.mp4,FAKE,chdbpzgrpy.mp4,train,dfdc_train_part_11
zqriqgfsnb.mp4,FAKE,onygmnjhph.mp4,train,dfdc_train_part_11
blruhiftcw.mp4,FAKE,bufydtkvxt.mp4,train,dfdc_train_part_11


In [11]:
fd=FaceDetector()

error: /root/opencv-3.4.0/modules/dnn/src/caffe/caffe_io.cpp:1145: error: (-2) FAILED: ReadProtoFromTextFile(param_file, param). Failed to parse NetParameter file: ../models/deploy.prototxt in function ReadNetParamsFromTextFileOrDie


In [23]:
nr = 3
nf = 4
faces = np.empty(shape=(0,32,32,3), dtype=np.int8)
labels = np.empty(shape=(0, 1), dtype="<U5")
for index, row in tqdm(df.sample(nr).iterrows(), total=nr):
    images = fd.extract_random_faces('../videos/' + row.dir + '/' + index, nf)
    if len(images) > 0:
        faces = np.append(faces,images,axis=0)
        labels = np.append(labels, np.full(shape=(len(images),1), fill_value=row.label, dtype="<U5"),axis=0)

100%|██████████| 3/3 [00:09<00:00,  4.15s/it]


In [24]:
assert len(faces) == len(labels)

In [26]:
np.savez('../data/train',faces=faces, labels=labels)